In [1]:
import mpoints.hybrid_hawkes_exp
import numpy as np

# Simulate

In [27]:
model = mpoints.hybrid_hawkes_exp.HybridHawkesExp(4,3,[0,1,2,3],[0,1,2])

In [28]:
model.transition_probabilities = np.random.uniform(size=model.transition_probabilities.shape)

In [85]:
model.set_hawkes_parameters(np.random.exponential(size=model.base_rates.shape), np.random.exponential(size=model.impact_coefficients.shape), 60*np.random.exponential(size=model.decay_coefficients.shape))

In [111]:
times, events, states = model.simulate(0,500)

In [112]:
len(times)

4631

# Fit

In [113]:
import torch

In [144]:
base_rates, impact_coefficients, initial_endo_intensity, decay_coefficients = [torch.tensor(np.random.exponential(size=x.shape),requires_grad=True) for x in (model.base_rates,model.impact_coefficients,model.impact_coefficients,model.decay_coefficients)]

In [145]:
optim = torch.optim.Adam(params=[base_rates,impact_coefficients,initial_endo_intensity,decay_coefficients])

In [146]:
import tqdm

In [154]:
for _ in tqdm.trange(1000):
    prev_t = 0
    ll = torch.tensor(0.0)
    endo_intensity = initial_endo_intensity.clone()
    optim.zero_grad()
    for t,e,s in zip(times[:100],events,states):
        time_diff = t-prev_t
        compensator += base_rates*time_diff
        endo_intensity *= torch.exp(-decay_coefficients*time_diff)
        endo_intensity[e,s,:] += impact_coefficients[e,s,:]
        ll += torch.log(base_rates[e]+endo_intensity[:,:,e].sum()) - (impact_coefficients/decay_coefficients).sum()
    (-ll).backward()
    optim.step()
    with torch.no_grad():
        base_rates.clamp_(0, None)
        base_rates.nan_to_num_(0)
        impact_coefficients.clamp_(0, None)
        impact_coefficients.nan_to_num_(0)
        initial_endo_intensity.clamp_(0, None)
        initial_endo_intensity.nan_to_num_(0)
        decay_coefficients.clamp_(impact_coefficients, None)
        decay_coefficients.nan_to_num_(0)
    prev_t = t

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:42<00:00, 23.44it/s]


In [167]:
impact_coefficients

tensor([[[0.0000, 0.2679, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 3.1689, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.9601, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000]]], dtype=torch.float64,
       requires_grad=True)

In [168]:
model.impact_coefficients

array([[[4.01818816e-01, 6.82134062e-01, 8.55629000e-01, 2.53865902e+00],
        [9.25906647e-01, 1.77952203e-01, 1.07726438e+00, 1.87435792e+00],
        [2.86680910e-01, 1.67919025e+00, 2.34928118e-01, 9.69451046e-03]],

       [[8.05238366e+00, 4.12738059e-01, 2.81014679e+00, 3.92482330e-02],
        [2.98992187e-02, 8.29021602e-01, 1.40681085e-01, 2.26714876e-01],
        [3.07772585e+00, 1.28036887e+00, 1.80525639e-01, 6.69377533e-02]],

       [[4.02470052e-02, 7.74943062e-01, 4.77077394e-03, 3.04805754e-01],
        [6.25886175e-01, 9.52707860e-01, 6.75667430e-01, 2.83536187e+00],
        [4.68970886e+00, 9.97236597e-01, 1.46157718e+00, 1.35527009e+00]],

       [[3.87901187e-01, 3.01839229e-01, 1.56068791e+00, 1.87257021e+00],
        [3.49368799e-01, 3.18099783e-02, 1.83247744e+00, 6.49485140e-01],
        [3.60863067e-01, 7.28575387e-01, 4.30658415e-01, 1.57415615e+00]]])

In [169]:
decay_coefficients

tensor([[[0.0000, 5.3991, 0.0000, 0.0000],
         [0.0000, 2.0459, 0.0000, 0.0000],
         [0.0000, 1.6002, 0.0000, 0.0000]],

        [[0.0000, 3.3141, 0.0000, 0.0000],
         [0.0000, 0.4304, 0.0000, 0.0000],
         [0.0000, 1.5612, 0.0000, 0.0000]],

        [[0.0000, 3.8327, 0.0000, 0.0000],
         [0.0000, 3.0938, 0.0000, 0.0000],
         [0.0000, 0.7980, 0.0000, 0.0000]],

        [[0.0000, 0.1053, 0.0000, 0.0000],
         [0.0000, 4.4116, 0.0000, 0.0000],
         [0.0000, 2.0006, 0.0000, 0.0000]]], dtype=torch.float64,
       requires_grad=True)

In [170]:
model.decay_coefficients

array([[[149.58260479,  37.1189172 ,  58.25088515,  91.89887187],
        [  4.14403012,  22.94247236,  26.42739271,   1.76351923],
        [ 23.27120181, 146.24042916,  19.28794975,  15.38577535]],

       [[ 26.39244469,  72.30012118,   2.0534134 ,  46.62901948],
        [ 41.73557367,  17.04206495,  36.05031098,  44.4459263 ],
        [ 10.03528571,  83.34770941,  76.04858073,  72.69321569]],

       [[  9.93609067,  41.580563  ,  27.93039715,   2.79479661],
        [170.87047453,  59.93485686,  17.04556129,  32.8890258 ],
        [ 64.42239462,  58.30625731,  23.260157  , 169.65730901]],

       [[150.5077581 , 132.51112832,  32.38837814,  48.72487767],
        [ 39.4058926 ,  71.57374197,   8.66052867,   1.34626895],
        [ 28.75292683,  99.89284754,  18.97569092,  10.69582698]]])

In [159]:
base_rates

tensor([0.0000, 2.2006, 0.0000, 0.0000], dtype=torch.float64,
       requires_grad=True)

In [160]:
model.base_rates

array([0.82556017, 0.44901124, 3.81648707, 1.83798344])